### Introduction to contravariant logging

We decided to use contravariant logging approach. I use here `co-log` lib naming convention because that is what we have in the code base now (`cardano-node` uses `contra-tracer`).

Some initial boilerplate imports and language extensions:

In [2]:
:ext GeneralizedNewtypeDeriving
:ext BlockArguments

import Control.Concurrent (threadDelay)

Let's define the core logging data type which is just a logging function `msg -> m ()` (for example `Text -> IO ()`). We wrap in a `newtype` and call it `LogAction`:

In [3]:
newtype LogAction m msg = LogAction (msg -> m ())

-- Let's define a handy runner which just applies the function.
log :: LogAction m msg -> msg -> m ()
log (LogAction l) = l

For example `putStrLn` can be easily turned into a log action and it is `String` logging action in `IO`:

In [4]:
consoleLogAction :: LogAction IO String
consoleLogAction = LogAction putStrLn

We can also easily implement a file logger (but please don't use it in production ;-):

In [5]:
fileLogAction :: FilePath -> LogAction IO String
fileLogAction filePath = LogAction \msg -> appendFile filePath (msg <> "\n")

#### Combining actions "veritically"

> Note: I used "vertically" in the title because we can imagine that we have a logging pipeline - `log entry -> log transformation / adjustment -> log consumers`. From the application point of view we have a single entry point to this pipeline.

If we have loggers which work with the same message type (`String` in our case) I think that we have pretty natural way of "combining" the logging actions - we are nearly able to use `deriving newtype` for that and use `->` and `m ()` underlying instances but let's write this composition by hand... I mean please implement `<>` and `mempty`: 

In [6]:
instance (Applicative m) => Semigroup (LogAction m msg) where
  LogAction log1 <> LogAction log2 = LogAction \msg -> log1 msg *> log2 msg
  
instance (Applicative m) => Monoid (LogAction m msg) where
  mempty = LogAction \msg -> pure ()

We should be able to compose multiple loggers now:

In [7]:
appLogAction = consoleLogAction <> fileLogAction "app.log"

So let's log something..

In [8]:
log appLogAction "Hello World!"

Hello World!

and check if both loggers have consumed the message: 

In [9]:
readFile "app.log" >>= putStrLn

Hello World!

#### "Horizontal" adjustments

##### Message type adaptation

Let's imagine that we want to attach pretty standard attribute to the log messages which are useful during debugging which is `Severity`. `co-log` provides this data type for us:

In [10]:
data Severity = Debug | Info | Warn | Error
  deriving (Eq, Ord, Show)

Now we can introduce a bit richer message structure plus some formatting helper:

In [11]:
data Message = Message { msgSeverity :: Severity, msgText :: String }

fmtMessage :: Message -> String
fmtMessage (Message severity msg) = "[" <> show severity <> "] " <> msg 

Can we use our previously defined `consoleLogAction` or `fileLogAction` but using this richer `Message` structure? Please try to write this adapter given `fmtMessage` function:

In [12]:
adaptStrLogAction :: LogAction m String -> LogAction m Message
adaptStrLogAction (LogAction logAction) = LogAction $ logAction . fmtMessage

appLogAction' :: LogAction IO Message
appLogAction' = adaptStrLogAction appLogAction

log appLogAction' (Message Debug "Adapted string logger test")

[Debug] Adapted string logger test

If we generalize the above adaptation API it forms a bit non intuitive interface which is called contravariant functor:

In [13]:
class Contravariant f where
  contramap :: (b -> a) -> f a -> f b

We can read the `contramap` operation semantics in our context like:

> Given a `LogAction m a` (let swe have logger which consumes a `String` at hand) and a function which turns `b` into `a` (we have `fmtMessage` which turns `Message` into `String`) we should be able to build a `LogAction m b` (in our case new `LogAction` gonna consume `Message` values).

Could you please implement `Contravariant` for `LogAction`:

In [38]:
instance Contravariant (LogAction m) where
  contramap f (LogAction logAction) = LogAction (logAction . f)
  
log (contramap fmtMessage consoleLogAction) (Message Debug "Adapted using contramap")

[Debug] Adapted using contramap

##### Logs context anotation

What is really interesting is that we can use the above API also to annotate the logs with context. Let's imagine that we have a subsystem entry point like:

```haskell
runSubsystem1 :: LogAction IO String -> IO ()
```

We can enhance the logger and pass it to the subsystem like:

```haskell
app = do
  appLogAction = consoleLogAction
  ...
  let
    subsystem1LogAction = contramap (\msg -> "[Subsystem]" <> msg) appLogAction
  runSubsystem subsystem1LogAction
```

If we imagine that we are representing log entries using app / subsystem specific type we could also use the same approach:

```
data AppLog = Subsystem1 ... | Subsystem2 ...

data Subsystem1Log = InitializationStarted | InitializationSucceeded | ...

runSubsystem1 :: LogAction IO Subsystem1Log -> IO ()
runSubsystem1 logAction = ...

appLogAction :: LogAction IO AppLog -> IO ()
appLogAction logAction = ...

app = do
  let
    subsystem1LogAction = contramap Subsystem1 appLogAction
  runSubsystem subsystem1LogAction
```
